# ⚛🧪 Molecule Data Aquired!

In this notebook, I will show you how to load and access the data for your chosen gasses!

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

### download the data from google drive

In [ ]:
! gdown https://drive.google.com/uc?id=1LK65H6dtv34R_evtIHZGFPJlvCDFdWA7 -O data.hdf5

Lets open the data up and check what it looks like

In [ ]:
ds = xr.open_dataset('data.hdf5')
ds

Some of the planets that we generated are not physical, this means that the spectra could not be correctly generated for at least some wavelengths.

This is represented in the data as `NaN` values. `NaN` stands for `N`ot `a` `N`umber, so the data point is invalid or missing.

We can convert these values into zeros now so that they don't mess with the experiment. Later on we might look at how to remove these values entirely.

In [ ]:
spectra = ds['spectra'].values
print(spectra.shape)

# Use numpy.isnan to check for NaN values
nan_count = np.isnan(spectra).sum()
print(f"Number of NaN values: {nan_count}")

# Remove NaN values by replacing them with 0 
spectra = np.nan_to_num(spectra)
print(spectra.shape)


## 📈 Plot some Spectra

Lets quickly plot some of the spectra to check that it looks as we were expecting!

We start by plotting just one value...

In [ ]:
plt.plot(ds['wavelength'], spectra[0], color='black') # Plot the first spectrum, the 0 here means the first, so we can change that to 1 to plot the second, or 2 for the 3rd etc

def plot_extras(): # Define a little function so that we don't have to repeat this code to mkae our plots look nicer
    plt.xscale('log')
    plt.xticks([1, 2, 3, 5, 7], ['1µm', '2µm', '3µm', '5µm', '7µm'])
    plt.xlabel('Wavelength (µm)')
    plt.ylabel('Tranit Depth (%)')

    plt.title('Simulated Exoplanet Transmission Spectra')
plot_extras() # Call the function we just defined

If we plot more than one spectra though we can begin to see that they are not very similar at all. This might not be the best for machine learning purposes.

In [ ]:
for i in range(1, 35): # Loop through the first 35 spectra
    plt.plot(ds['wavelength'], spectra[i], alpha=0.5) # Plot the spectra with a different colour, and a little transparency, this is what alpha=0.5 does. alpha=1 is fully opaque, and alpha=0 is fully transparent

plot_extras()

## 🔧 **Normalization**
Normalization scales the data to a smaller, consistent range—usually between 0 and 1 or -1 and 1. It ensures that spectra with different scales contribute equally to the model.

### Why Normalize? 
- 🏋️‍♀️ **Balances Feature Contribution**: Prevents features with larger ranges (outliers) from dominating the learning process.
- 🚀 **Improves Convergence**: Helps algorithms converge faster.
- 🧠 **Enhances Model Accuracy**: Leads to better-performing models in many cases.

### Common Normalization Techniques:
1. **Min-Max Scaling**:

   $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$

   Scales features to a range of 0 to 1.
   
2. **Z-Score Standardization**: 
   
   $x' = \frac{x - \mu}{\sigma}$

   Centers the data around 0 with a standard deviation of 1.

---

### ✨ Hint: **Division by ZERO**

When carrying out scaling, we have no guarantee about the values which are going to be fed through the equation. If there is a case where we think that a value might go to zero, we can add a very small value ` + np.finfo(float).eps`, which is equal to $1\times10^{-16}$, to prevent dividing by that number from causing an error!

---

Lets try and impliment some min max scaling...

In [ ]:

def min_max_normaliser(x, dim=None):
    '''normalises the input np array to the range [0, 1]'''

    x = np.array(x)
    x_min = np.min(x, axis=dim, keepdims=True)
    x_max = np.max(x, axis=dim, keepdims=True)
    
    x = x # <---- your code here...
    
    return x

lets plot some of the normalised spectra...

What does this look like? What should this look like? 

In [ ]:
for i in range(1, 35):
    plt.plot(ds['wavelength'], min_max_normaliser(spectra, dim=1)[i], alpha=0.5)

plot_extras()

We can now load our labels, the actual values of abundance that we are training against

In [ ]:
mol_header = ds['molecules'].values
chemical_abundances = ds['log_abundance'].values

print(mol_header)
print(chemical_abundances.shape)

We don't want to train a model that predicts all of these gasses at once, so we just choose the one/ones that we are interested in, for example here $H_2O$ and $CH_4$

In [ ]:
included_molecules = ['H2O', 'CH4']
index = np.isin(mol_header, included_molecules)

labels = chemical_abundances[:, index]
labels.shape

## 📝 Test and Train Data!

While we could then train our model on all the data, because we are looking to validate the performance of the models, we hold back some of the data that the model has not seen before so that we can evaluate its performance! We will use 20\% of the data for testing, and train on the rest

In [ ]:
# split train into train and test - we will use the test data to evaluate the model

tts = [0.8, 0.2] # 80% train, 20% test
seed = 42 # random seed for reproducibility
train_spectra, test_spectra, train_labels, test_labels = train_test_split(
    spectra, # here we input the regular spectra, but we should probably use the normalised spectra
    labels, 
    test_size=tts[1], 
    random_state=seed)

# print shapes in table
print(f"""
train_spectra: {train_spectra.shape}
train_labels: {train_labels.shape}

test_spectra: {test_spectra.shape}
test_labels: {test_labels.shape}
""")

---

## 🤖 Model: 

Define your best model from previous sessions here! This is just an example

In [ ]:
class model_A(nn.Module):
    def __init__(self):
        super(model_A, self).__init__()

        self.fc1 = # Your code here...

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = # Your code here...
        return x

In [ ]:
model = # Your code here...
lr = # Your code here...
batch_size = # Your code here...

# create a loss function
criterion = nn.MSELoss()


# use GPU if available for faster training times
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Using GPU')
elif torch.mps.is_available():
    device = torch.device("mps")
    print('Using MPS')
else:
    device = torch.device("cpu")
    print('Using CPU')

model.to(device)
test_spectra = torch.Tensor(test_spectra).to(device)
test_labels = torch.Tensor(test_labels).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
train_dataset = TensorDataset(torch.Tensor(train_spectra).to(device), torch.Tensor(train_labels).to(device))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
batch_losses = []
epoch_train_losses = []
epoch_test_losses = []

# train the model
model.train()
for epoch in range(50): # Keep the batch size low so we can train this quickly for today...
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        batch_losses.append(loss.item())

    with torch.no_grad():
        test_outputs = model(torch.Tensor(test_spectra))
        test_loss = criterion(test_outputs, test_labels)

    epoch_test_losses.append(test_loss.item())
    epoch_train_losses.append(np.mean(batch_losses))
    batch_losses = []

    if epoch % 20 == 0:
        print(f'epoch {epoch}, train loss {epoch_train_losses[-1]}, test loss {test_loss.item()}')

---

## ✅ Validation:

We will use our test data that the model has never seen in order to test how accurate the model is with it's predictions. Then we can create some plots the same as last time...

In [ ]:
# generate the test set predictions
with torch.no_grad():
    test_outputs = model(test_spectra)

# plot the final predictions
for i in range(labels.shape[1]):
    plt.figure(figsize=(5, 5))
    plt.plot(test_labels.cpu().numpy()[:, i], test_outputs.cpu().numpy()[:, i],'kx', label=mol_header[i], alpha=0.5)

    plt.plot([test_labels.cpu().numpy()[:, i].min(), 
            test_labels.cpu().numpy()[:, i].max()], 
            [test_labels.cpu().numpy()[:, i].min(), 
            test_labels.cpu().numpy()[:, i].max()], 'r--', lw=2)

    plt.xlabel(f'True Abundance of {mol_header[index][i]}')
    plt.ylabel(f'Predicted Abundance of {mol_header[index][i]}')


Does this look good? Is the model performing as expected? If not, consider, is the normalisation working as anticipated? Are there different ways that you could normalise the data? 

We can print a quick overview of what we have achieved here

In [ ]:
print(f'best test loss: {min(epoch_test_losses)}')
print('---')
print(f'model architecture: {model}')
print(f'learning rate: {lr}')
print(f'batch size: {batch_size}')

---

# 🌱 **Data Augmentation**
Could these results be better? If we had more data...? Augmentation is the process of artificially increasing the size of your dataset by creating modified versions of existing data. 

### Why Augment? 
- 📈 **Boosts Dataset Size**: Helps when data is limited.
- 🔄 **Introduces Variability**: Makes the model more robust to unseen data.
- 🏆 **Reduces Overfitting**: Prevents the model from memorizing the training data.

### Examples of Augmentation in Regression:
- **Noise Injection**: Add a small amount of random noise to input features.
- **Synthetic Data Creation**: Generate new samples using domain knowledge or interpolation between existing samples.

---


Can we double our dataset size by creating noisy version of the data? We know how to create random values like this:

In [ ]:
random_spectra = np.random.rand(52)

plt.plot(ds['wavelength'], random_spectra, color='black')
plot_extras()

Obviously this on it's own is meaningless, but is there a way to combine this with the existing data, then we could use it for training.....

In [ ]:
# Your code here...